In [1]:
import requests
import pandas as pd

In [7]:
# API configuration
url = "https://zillow-working-api.p.rapidapi.com/search/byaddress"
headers = {
    "X-RapidAPI-Host": "zillow-working-api.p.rapidapi.com",
    "X-RapidAPI-Key": "8d455c96b8msh3a9f43cf0c88dc4p11cc38jsnc4b82e3c5c66"
}

# Query parameters
params = {
    "location": "VA",
    #"page": "1",
    "sortOrder": "Newest",
    "listingStatus": "Sold"
}


In [8]:
# Function to fetch data from the API
def fetch_data(params):
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# Fetch data
data = fetch_data(params)
print(data)
# Process the data
if data and 'results' in data:
    properties = data['results']
    
    # Extract relevant information
    extracted_data = []
    for prop in properties:
        extracted_data.append({
            'Address': prop.get('streetAddress', ''),
            'City': prop.get('city', ''),
            'State': prop.get('state', ''),
            'Zip': prop.get('zipcode', ''),
            'Price': prop.get('price', ''),
            'Bedrooms': prop.get('bedrooms', ''),
            'Bathrooms': prop.get('bathrooms', ''),
            'Living Area': prop.get('livingArea', ''),
            'Lot Size': prop.get('lotSize', ''),
            'Year Built': prop.get('yearBuilt', ''),
            'Days on Zillow': prop.get('daysOnZillow', ''),
            'Zestimate': prop.get('zestimate', '')
        })

    # Create a DataFrame
    df = pd.DataFrame(extracted_data)

    # Write to CSV
    csv_filename = "VA_Sold_Properties.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Data has been written to {csv_filename}")
else:
    print("No data found or error in API response")

{'message': '200', 'source': '7or_ws_ncch', 'resultsCount': {'totalMatchingCount': 409134, 'ungroupedResultCount': 0, 'scrapeable_count': 1000}, 'pagesInfo': {'totalPages': 5.0, 'currentPage': 1, 'resultsPerPage': 200}, 'searchResults': [{'property': {'zpid': 12047481, 'location': {'latitude': 38.81619, 'longitude': -77.12412}, 'address': {'streetAddress': '5340 Holmes Run Pkwy UNIT 1609', 'zipcode': '22304', 'city': 'Alexandria', 'state': 'VA', 'buildingId': 1001728020}, 'media': {'propertyPhotoLinks': {'mediumSizeLink': 'https://photos.zillowstatic.com/fp/e841584c72f5b3fbb03ef01659b46e3a-p_c.jpg', 'highResolutionLink': 'https://photos.zillowstatic.com/fp/e841584c72f5b3fbb03ef01659b46e3a-p_f.jpg'}, 'thirdPartyPhotoLinks': {}, 'hasVRModel': False, 'hasVideos': False, 'hasApprovedThirdPartyVirtualTour': False, 'allPropertyPhotos': {'medium': ['https://photos.zillowstatic.com/fp/e841584c72f5b3fbb03ef01659b46e3a-p_c.jpg'], 'highResolution': ['https://photos.zillowstatic.com/fp/e841584c72f

In [12]:
from datetime import datetime
# Process the data
if data and 'searchResults' in data:
    properties = [item['property'] for item in data['searchResults'] if 'property' in item]
    
    # Extract relevant information
    extracted_data = []
    for prop in properties:
        last_sold_date = prop.get('lastSoldDate', '')
        if last_sold_date:
            last_sold_date = datetime.fromtimestamp(last_sold_date / 1000).strftime('%Y-%m-%d')

        # Calculate lot size in square feet if the unit is in acres
        lot_size_acres = prop.get('lotSizeWithUnit', {}).get('lotSize', '')
        lot_size_unit = prop.get('lotSizeWithUnit', {}).get('lotSizeUnit', '')
        lot_size_sqft = ''
        if lot_size_acres and lot_size_unit.lower() == 'acres':
            lot_size_sqft = float(lot_size_acres) * 43560  # Convert acres to square feet
        extracted_data.append({
            'Address': prop['address'].get('streetAddress', ''),
            'City': prop['address'].get('city', ''),
            'State': prop['address'].get('state', ''),
            'Zip': prop['address'].get('zipcode', ''),
            'Price': prop['price'].get('value', '') if 'price' in prop else '',
            'pricePerSquareFoot': prop['price'].get('pricePerSquareFoot', '') if 'price' in prop else '',
            'Bedrooms': prop.get('bedrooms', ''),
            'Bathrooms': prop.get('bathrooms', ''),
            'Living Area': prop.get('livingArea', ''),
            'Lot Size (Acres)': lot_size_acres,
            'Lot Size (Sqft)': lot_size_sqft,  # Add calculated field
            'Year Built': prop.get('yearBuilt', ''),
            'Days on Zillow': prop.get('daysOnZillow', ''),
            'Zestimate': prop['estimates'].get('zestimate', '') if 'estimates' in prop else '',
            'Rent Zestimate': prop['estimates'].get('rentZestimate', '') if 'estimates' in prop else '',
            'Last Sold Date': last_sold_date,
            'Property Type': prop.get('propertyType', ''),
            'Listing Status': prop['listing'].get('listingStatus', '') if 'listing' in prop else ''
        })

    # Create a DataFrame
    df = pd.DataFrame(extracted_data)

    # Write to CSV
    csv_filename = "VA_Sold_Properties.csv"
    df.to_csv(csv_filename, index=False)
    print(f"Data has been written to {csv_filename}")
else:
    print("No data found or error in API response")

df.head(4)

Data has been written to VA_Sold_Properties.csv


,Address,City,State,Zip,Price,pricePerSquareFoot,Bedrooms,Bathrooms,Living Area,Lot Size (Acres),Lot Size (Sqft),Year Built,Days on Zillow,Zestimate,Rent Zestimate,Last Sold Date,Property Type,Listing Status
0,5340 Holmes Run Pkwy UNIT 1609,Alexandria,VA,22304,189000,338,1,1.0,560,,,1967,1,183000,1704,2025-01-31,condo,recentlySold
1,279 S Pickett St APT 402,Alexandria,VA,22304,495000,322,2,2.0,1535,,,1995,1,492800,2898,2025-01-31,condo,recentlySold
2,2792 N Quebec St,Arlington,VA,22207,1667500,654,5,4.0,2550,0.46219,20133.0,1967,1,1732200,6381,2025-01-31,singleFamily,recentlySold
3,1110 N Inglewood St,Arlington,VA,22205,1136110,664,3,2.0,1710,5438.0,,1941,1,1028500,3635,2025-01-31,singleFamily,recentlySold


In [15]:
import json
import csv
from collections import OrderedDict

def flatten_json(nested_json, prefix=''):
    flattened = OrderedDict()
    for key, value in nested_json.items():
        new_key = f"{prefix}.{key}" if prefix else key
        if isinstance(value, dict):
            flattened.update(flatten_json(value, new_key))
        elif isinstance(value, list):
            for i, item in enumerate(value):
                if isinstance(item, dict):
                    flattened.update(flatten_json(item, f"{new_key}[{i}]"))
                else:
                    flattened[f"{new_key}[{i}]"] = item
        else:
            flattened[new_key] = value
    return flattened

# Read JSON from file
with open('rentcat_marketdata_json.txt', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# Flatten the JSON
flattened_data = flatten_json(json_data)

# Write to CSV
with open('rentcat_marketdata_json.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Key', 'Value'])  # Header
    for key, value in flattened_data.items():
        writer.writerow([key, value])

print("CSV file 'rentcat_marketdata_json.csv' has been created with the expanded key-value pairs.")


CSV file 'rentcat_marketdata_json.csv' has been created with the expanded key-value pairs.
